In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import datetime

from tqdm import tqdm
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
from sklearn.metrics import mean_squared_error

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

### Data Load

In [2]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
df_hist = utils.read_df_pkl('../input/hist*0*')
df_new = utils.read_df_pkl('../input/new*0*')

100%|██████████| 3/3 [00:01<00:00,  2.38it/s]


In [3]:
def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

In [4]:
for df in [df_hist, df_new]:
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['weekofyear'] = df['purchase_date'].dt.weekofyear
    df['month'] = df['purchase_date'].dt.month
    df['hour'] = df['purchase_date'].dt.hour
    df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
    df['date_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)

In [5]:
df_hist.drop(['city_id', 'category_1', 'installments', 'category_2', 'category_3', 'merchant_category_id', 'merchant_id', 'purchase_amount'], axis=1, inplace=True)
df_new.drop(['city_id', 'category_1', 'installments', 'category_2', 'category_3', 'merchant_category_id', 'merchant_id', 'purchase_amount'], axis=1, inplace=True)

df_hist['yyyymm'] =  df_hist['purchase_date'].map(lambda x: str(x)[:7])
df_new['yyyymm'] =  df_new['purchase_date'].map(lambda x: str(x)[:7])
df_hist['yyyymm_week'] = df_hist['yyyymm'] + df_hist['weekofyear'].map(lambda x: '-' + str(x))
df_new['yyyymm_week'] = df_new['yyyymm'] + df_new['weekofyear'].map(lambda x: '-' +  str(x))
df_hist.head()

,authorized_flag,card_id,month_lag,purchase_date,state_id,subsector_id,weekofyear,month,hour,month_diff,date_diff,yyyymm,yyyymm_week
0,Y,C_ID_4e6213e9bc,-8,2017-06-25 15:33:07,16,37,25,6,15,18,554,2017-06,2017-06-25
1,Y,C_ID_4e6213e9bc,-7,2017-07-15 12:10:45,16,16,28,7,12,17,535,2017-07,2017-07-28
2,Y,C_ID_4e6213e9bc,-6,2017-08-09 22:04:29,16,37,32,8,22,16,509,2017-08,2017-08-32
3,Y,C_ID_4e6213e9bc,-5,2017-09-02 10:06:26,16,34,35,9,10,16,486,2017-09,2017-09-35
4,Y,C_ID_4e6213e9bc,-11,2017-03-10 01:14:19,16,37,10,3,1,22,662,2017-03,2017-03-10


### 連続week, month利用回数、期間内week, monthカバー率を特徴にする

In [31]:
if hist_new_name=='hist':
    fname = '130_ddf'
elif hist_new_name=='new':
    fname = '131_ddf'
ignore_features = ['first_active_month', 'card_id']

for col in df_train.columns:
#     if col in ignore_features:
#         continue
#     if not(col.count('feature_')):continue
    if not(col.count('date_diff')):continue
#     if not(col.count('trans')):continue
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_train_{col}@', obj=df_train[col].values)
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_test_{col}@', obj=df_test[col].values)

### 132 date_diffの前半、後半における違いを特徴にする. 全体verも